In [1]:
import sys
import os

paths_to_add = [
    '/root',         
    '/root/task'   
]

for p in paths_to_add:
    if p not in sys.path:
        sys.path.append(p)
        print(f"成功加入路径: {p}")
        
from src.YoloSAM.utils.mask_to_yolo import MaskToYOLOConverter
from src.YoloSAM.utils.config import YOLOConfig
from src.YoloSAM.scripts.train_yolo import YOLOTrainer

converter = MaskToYOLOConverter(class_id=0)
converter.convert_dataset_inplace(
    base_path='../../../datasets/DRIVE',
    min_area=5,
    splits=['train', 'val']
)

config = YOLOConfig(
    # Model Settings
    model_type="yolo11n",
    device='cuda',
    pretrained_path='../checkpoints',

    # DataSet Settings
    dataset_path='../../../datasets/DRIVE/train',
    val_dataset_path='../../../datasets/DRIVE/val',
    class_names=['scar'],

    # Training parameters
    epochs=10,
    batch_size=16,
    image_size=640,
    patience=50,

    # Augmentation (optimized for medical scars)
    mosaic=0.9,
    mixup=0.1,
    copy_paste=0.4,
    degrees=15.0,
    hsv_v=0.3,

    # Detection parameters
    iou_threshold=0.2,
    conf_threshold=0.15,
    max_detections=2,

    # Project settings
    project_name="yolo_scar_detection",
    experiment_name="enhanced_scar_detection",

    # Wandb settings
    wandb_project="YOLO-scar-detection",
    wandb_name="scar_detection_v1",
    wandb_mode="disabled"
)

trainer = YOLOTrainer(config)
results = trainer.train()


成功加入路径: /root
成功加入路径: /root/task
Processing train split...
Found 100 mask files in train


Converting train: 100%|██████████| 100/100 [00:00<00:00, 402.47it/s]


Completed train split: 100 files processed
Labels saved to: ../../../datasets/DRIVE/train/labels
Processing val split...
Found 20 mask files in val


Converting val: 100%|██████████| 20/20 [00:00<00:00, 423.17it/s]


Completed val split: 20 files processed
Labels saved to: ../../../datasets/DRIVE/val/labels
Dataset conversion completed!
Your dataset structure is now:
  ../../../datasets/DRIVE/train/
    images/
    masks/
    labels/  <- NEW
  ../../../datasets/DRIVE/val/
    images/
    masks/
    labels/  <- NEW
Loading pretrained YOLO model: yolo11n
Created dataset configuration at: ../../../datasets/DRIVE/dataset.yaml
Starting YOLO training for 10 epochs
Starting YOLO training with configuration:
  data: ../../../datasets/DRIVE/dataset.yaml
  epochs: 10
  imgsz: 640
  batch: 16
  patience: 50
  box: 7.5
  cls: 0.5
  dfl: 1.5
  iou: 0.2
  max_det: 2
  mosaic: 0.9
  mixup: 0.1
  copy_paste: 0.4
  scale: 0.5
  fliplr: 0.5
  flipud: 0.1
  degrees: 15.0
  translate: 0.3
  hsv_h: 0.0
  hsv_s: 0.0
  hsv_v: 0.3
  lr0: 0.001
  lrf: 0.0001
  warmup_epochs: 10
  weight_decay: 0.001
  save_period: 50
  project: runs
  name: yolo_scar_detection
  multi_scale: True
  device: cuda
Ultralytics 8.3.237 🚀 Python